In [1]:
# from Recommenders.Hybrid.SimilarityMergedHybridRecommender import SimilarityMergedHybridRecommender
# from src.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaCBFRecommender import RP3betaCBFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDItemRecommender
from Recommenders.Hybrid.GeneralizedMergedHybridRecommender import (
    GeneralizedMergedHybridRecommender,
)
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_ElasticNet
from Recommenders.Implicit.FeatureCombinedImplicitALSRecommender import (
    FeatureCombinedImplicitALSRecommender,
)
from Recommenders.Hybrid.GeneralizedSimilarityMergedHybridRecommender import (
    GeneralizedSimilarityMergedHybridRecommender,
)
from Utils.ICM_preprocessing import *
from Utils.data_loader import load_URM, load_ICM
from Utils.confidence_scaling import *
from Utils.write_submission import write_submission
from Data_manager.split_functions.split_train_validation_random_holdout import (
    split_train_in_two_percentage_global_sample,
)
from Evaluation.K_Fold_Evaluator import K_Fold_Evaluator_MAP
from bayes_opt import BayesianOptimization

%env OPENBLAS_NUM_THREADS=1


env: OPENBLAS_NUM_THREADS=1


In [2]:
URM_all = load_URM("input/data_train.csv")
ICM_all = load_ICM("input/data_ICM_subgenre.csv")

# ICM_combined = combine(ICM=ICM_all, URM = URM_all)


In [3]:
URMs_train = []
URMs_validation = []

for k in range(5):
    URM_train, URM_validation = split_train_in_two_percentage_global_sample(
        URM_all, train_percentage=0.80
    )
    URMs_train.append(URM_train)
    URMs_validation.append(URM_validation)


In [4]:
evaluator_validation = K_Fold_Evaluator_MAP(
    URMs_validation, cutoff_list=[10], verbose=False
)

ICMs_combined = []
for URM in URMs_train:
    ICMs_combined.append(combine(ICM=ICM_all, URM=URM))


In [5]:
IALS_recommenders = []
rp3betaCBF_recommenders = []
userRP3beta_recommenders = []

for index in range(len(URMs_train)):
    IALS_recommenders.append(
        FeatureCombinedImplicitALSRecommender(
            URM_train=URMs_train[index], ICM_train=ICM_all, verbose=True
        )
    )

    # TODO: train this again by itself
    IALS_recommenders[index].fit(
        factors=int(250),
        regularization=0.01,
        use_gpu=False,
        iterations=43,
        num_threads=4,
        confidence_scaling=linear_scaling_confidence,
        **{"URM": {"alpha": 50}, "ICM": {"alpha": 50}}
    )

    rp3betaCBF_recommenders.append(
        RP3betaCBFRecommender(
            URM_train=URMs_train[index], ICM_train=ICMs_combined[index], verbose=False
        )
    )

    # TODO: train this again by itself
    rp3betaCBF_recommenders[index].fit(
        topK=int(529.1628484087545),
        alpha=0.45304737831676245,
        beta=0.226647894170121,
        implicit=False,
    )

    # userRP3beta_recommenders.append(
    #     UserRP3betaRecommender(
    #         URM_train=ICMs_combined[index].T,
    #         verbose=False
    #     )
    # )

    # userRP3beta_recommenders[index].fit(
    #     topK=int(252.2),
    #     alpha=0.2658,
    #     beta=0.2847,
    #     implicit=False
    # )




100%|██████████| 1/1 [00:12<00:00, 12.38s/it]


## Tuning hybrid weights

In [6]:
tuning_params = {
    "hybridWeight1":  (0, 1),
    "hybridWeight2":  (0, 1),
    # "hybridWeight3": (0, 1)
}

results = []

def BO_func(
    hybridWeight1,
    hybridWeight2
    # hybridWeight3
):
    recommenders = []

    for index in range(len(URMs_train)):
        recommender = GeneralizedMergedHybridRecommender(
            URM_train=URMs_train[index],
            recommenders=[
                IALS_recommenders[index],
                rp3betaCBF_recommenders[index],
                # userRP3beta_recommenders[index],
            ],
            verbose=True,
        )

        totalWeight = hybridWeight1 + hybridWeight2  # +hybridWeight3
        recommender.fit(
            alphas=[
                hybridWeight1 / totalWeight,
                hybridWeight2 / totalWeight
                # hybridWeight3/totalWeight
            ]
        )

        recommenders.append(recommender)

    result = evaluator_validation.evaluateRecommender(recommenders)
    results.append(result)

    return sum(result) / len(result)

In [7]:
optimizer = BayesianOptimization(
    f=BO_func,
    pbounds=tuning_params,
    verbose=5,
    random_state=5,
)

optimizer.maximize(
    init_points=30,
    n_iter=50,
)

|   iter    |  target   | hybrid... | hybrid... |
-------------------------------------------------
[0.1891604357403375, 0.19181556851264392]
|  1        |  0.1905   |  0.222    |  0.8707   |
[0.1891331249185879, 0.19179962512837767]
|  2        |  0.1905   |  0.2067   |  0.9186   |
[0.18941306302219937, 0.19199663045892898]
|  3        |  0.1907   |  0.4884   |  0.6117   |
[0.18955763330635003, 0.19220810201904873]
|  4        |  0.1909   |  0.7659   |  0.5184   |
[0.18957024344516907, 0.19218835525221434]
|  5        |  0.1909   |  0.2968   |  0.1877   |
[0.18907978551424467, 0.19170508431994368]
|  6        |  0.1904   |  0.08074  |  0.7384   |
[0.18971671818883773, 0.1924341648434852]
|  7        |  0.1911   |  0.4413   |  0.1583   |
[0.1896612533495221, 0.19246984917901633]
|  8        |  0.1911   |  0.8799   |  0.2741   |
[0.18952886288356588, 0.19219833186199334]
|  9        |  0.1909   |  0.4142   |  0.2961   |
[0.18943214832337685, 0.1921450437970548]
|  10       |  0.1908   |

In [23]:
best = optimizer.max
print("TARGET MAP", best["target"])

hw1 = best["params"]["hybridWeight1"]
hw2 = best["params"]["hybridWeight2"]

print("HYBRID WEIGHTS", hw1, hw2)

totalWeight = hw1 + hw2
totalWeight

alpha1, alpha2 = hw1 / totalWeight, hw2 / totalWeight

print("ALPHAS", alpha1, alpha2)

TARGET MAP 0.19107544151616146
HYBRID WEIGHTS 0.4413092228959531 0.1583098677126512
ALPHAS 0.7359826093062363 0.26401739069376373


## Train final model with tuned hyperparameters

In [25]:
recommender = GeneralizedMergedHybridRecommender(
    URM_train=URMs_train[0], # maybe we can take a better option instead of taking the first one of everything?
    recommenders=[
        IALS_recommenders[0],
        rp3betaCBF_recommenders[0],
        # userRP3beta_recommenders[index],
    ],
    verbose=True,
)

recommender.fit(
    alphas=[
        alpha1,
        alpha2
        # alpha3
    ]
)


In [34]:
write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
                    out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

In [35]:
print('output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))

output/FeatureCombinedImplicitALSRP3betaCBFHybridRecommender_submission.csv


In [26]:
# p3alpha_recommender = P3alphaRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# p3alpha_recommender.fit(
#     topK=int(212.8832860130684),
#     alpha=0.4729294763382114,
#     implicit=True
# )


In [27]:
# RP3 BETA (Combined) -----------------------------------------

# rp3betaCombined_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_combined,
#     verbose=False
# )

# rp3betaCombined_recommender.fit(
#     topK=int(529.1628484087545),
#     alpha=0.45304737831676245,
#     beta=0.226647894170121,
#     implicit=True
# )


In [28]:
# IALS -----------------------------------------

# IALS_recommender= FeatureCombinedImplicitALSRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=True
# )

# IALS_recommender.fit(
#     factors=int(398.601583855084),
#     regularization=0.01,
#     use_gpu=False,
#     iterations=int(94.22855449116447),
#     num_threads=6,
#     confidence_scaling=linear_scaling_confidence,
#     **{
#         'URM': {"alpha": 42.07374324671451},
#         'ICM': {"alpha": 41.72067133975204}
#     }
# )


In [29]:
# SLIM -----------------------------------------

# SLIM_recommender = MultiThreadSLIM_ElasticNet(
#         URM_train=ICM_combined.T,
#         verbose=False
#     )

# SLIM_recommender.fit(
#     alpha=0.00026894910579512645,
#     l1_ratio=0.08074126876487486,
#     topK=int(395.376118479588),
#     workers=6
# )

# SLIM_recommender.URM_train = URM_all


In [30]:
# OTHER STUFF ----------------------------------

# rp3betaCBF_recommender= RP3betaCBFRecommender(
#     URM_train=URM_all,
#     ICM_train=ICM_all,
#     verbose=False
# )
#
# rp3betaCBF_recommender.fit(
#     topK=int(117.1),
#     alpha=0.9882,
#     beta=0.7703,
#     implicit=False
# )

# itemKNN_recommender= ItemKNNCFRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
# itemKNN_recommender.fit(
#     topK=100,
#     shrink=50
# )
#
# pureSVD_recommender= PureSVDItemRecommender(
#     URM_train=URM_all,
#     verbose=False
# )
#
#
# pureSVD_recommender.fit(
#     num_factors=772,
#     topK= 599
# )


In [31]:
# Generalized Merged Hybrid -----------------------------------------

# recommender = GeneralizedMergedHybridRecommender(
#     URM_train=URM_all,
#     recommenders=[
#         IALS_recommender,
#         rp3betaCombined_recommender
#         #SLIM_recommender
#     ],
#     verbose=False
# )


In [32]:
# recommender.fit(
#     alphas=[
#         0.6836750866517823,
#         0.45969038157844144,
#         #0.2723405593515382
#     ]
# )


In [33]:
# write_submission(recommender=recommender, target_users_path="input/data_target_users_test.csv",
#                     out_path='output/{}_submission.csv'.format(recommender.RECOMMENDER_NAME))
